In [0]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(42)

## Collect Data
Download data from Project Gutenberg site -> http://www.gutenberg.org/files/1342/1342-0.txt

In [14]:
book_text = open('1342-0.txt', encoding='utf8').read()
print('Length of the book: ', len(book_text))

Length of the book:  775741


In [24]:
print(book_text[0:10])

﻿
The Proj


## Tokenize the data

In [4]:
t = tf.keras.preprocessing.text.Tokenizer(char_level = True)
t.fit_on_texts(book_text)
vocab_size = len(t.word_index)
print('Number of unique characters: ', vocab_size)

Number of unique characters:  66


In [16]:
t.word_index

{'\n': 15,
 ' ': 1,
 '!': 36,
 '#': 59,
 '$': 63,
 '%': 66,
 "'": 61,
 '(': 46,
 ')': 47,
 '*': 49,
 ',': 22,
 '-': 38,
 '.': 24,
 '/': 51,
 '0': 48,
 '1': 41,
 '2': 42,
 '3': 43,
 '4': 44,
 '5': 45,
 '6': 50,
 '7': 54,
 '8': 52,
 '9': 53,
 ':': 40,
 ';': 29,
 '?': 37,
 '@': 62,
 '[': 58,
 ']': 60,
 '_': 32,
 'a': 4,
 'b': 21,
 'c': 16,
 'd': 11,
 'e': 2,
 'f': 19,
 'g': 20,
 'h': 8,
 'i': 6,
 'j': 30,
 'k': 26,
 'l': 12,
 'm': 14,
 'n': 7,
 'o': 5,
 'p': 23,
 'q': 35,
 'r': 10,
 's': 9,
 't': 3,
 'u': 13,
 'v': 25,
 'w': 18,
 'x': 33,
 'y': 17,
 'z': 31,
 'à': 57,
 'é': 65,
 'ê': 56,
 '—': 39,
 '‘': 55,
 '’': 34,
 '“': 27,
 '”': 28,
 '\ufeff': 64}

In [0]:
#convert characters in the book to Numbers
book_num = t.texts_to_sequences(book_text)

In [26]:
print(book_num[0:10])

[[64], [15], [3], [8], [2], [1], [23], [10], [5], [30]]


In [0]:
#Build a dictionary which can convert numbers into chars
int_to_char = dict((i,c) for c, i in t.word_index.items())

## Batch Generator


In [0]:
import numpy as np

In [0]:
record_num = 0
sequence_length = 100
def batch_generator(batch_size=32):
  global record_num
  while True:
    input_data = []
    output_data = []

    for i in range(batch_size):
      input_seq = book_num[(record_num * sequence_length):(record_num * sequence_length)+sequence_length]
      output_seq = book_num[(record_num * sequence_length)+sequence_length]
      
      input_data.append(input_seq)
      output_data.append(output_seq)

      record_num = record_num + 1

      if((record_num*sequence_length + sequence_length) > len(book_num)):
        record_num = 0
    
    input_data = tf.keras.utils.to_categorical(input_data,num_classes=vocab_size+1)
    #Output data one hot encoding
    output_data = tf.keras.utils.to_categorical(output_data,num_classes=vocab_size+1)
    #Reshape input data into 3 dimensional numpy array
    #batch_size x sequence_length x vocab_size+1
    input_data = np.reshape(input_data,(len(input_data),sequence_length,vocab_size+1))
    yield input_data, output_data

## Build a model

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(sequence_length,vocab_size+1)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(vocab_size+1, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy')

## Print model output during training

In [0]:
start_pos = np.random.randint(0, high=(len(book_num)-sequence_length))
test_seq = book_num[start_pos: start_pos + sequence_length]

In [38]:
print('Initial sequence is: ')
for i in range (sequence_length):
  print(int_to_char[test_seq[i][0]], end='')

Initial sequence is: 
n the place, and his intrigues,
      all honoured with the title of seduction, had been extended in

## Prediction function

In [0]:
def predict_seq(epoch, logs):
  print('\n\nOutput sequence after epoch ', epoch, ' :')
  
  #Initialize predicted output
  predicted_output = ''
  
  #lets predict 50 next chars
  current_seq = np.copy(test_seq)
  
  for i in range(50):
    current_seq_one_hot = tf.keras.utils.to_categorical(current_seq, num_classes=vocab_size+1)
    data_input = np.reshape(current_seq_one_hot,(1,current_seq_one_hot.shape[0],current_seq_one_hot.shape[1]))
    print(data_input.shape)
    #Get the char int with maximum probability
    predicted_char_int = np.argmax(model.predict(data_input)[0])
    
    #Add to the predicted out, convert int to char
    predicted_output = predicted_output + int_to_char[predicted_char_int]
    
    #Update seq with new value at the end
    current_seq = np.roll(current_seq, -1)
    current_seq[current_seq.shape[0]-1] = [predicted_char_int]
  
  print(predicted_output)

In [0]:
#Create a LabdaCallback to do prediction at end of every epoch
lambda_checkpoint = tf.keras.callbacks.LambdaCallback(on_epoch_end=predict_seq)
#Create a model checkpoint to store model after each epoch if loss reduces
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('char_rnn.h5',monitor='loss',save_best_only=True, verbose = 1)

In [42]:
batch_size = 64
train_generator = batch_generator(batch_size=batch_size)
model.fit_generator(train_generator, epochs=1, steps_per_epoch=(len(book_num)-sequence_length)//batch_size, callbacks=[model_checkpoint, lambda_checkpoint])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
12118/12119 [============================>.] - ETA: 0s - loss: 0.6809
Epoch 00001: loss improved from inf to 0.68086, saving model to char_rnn.h5


Output sequence after epoch  0  :
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
(1, 100, 67)
selding the  and and that im an at she tommnthon t
12119/12119 [==========